## Factors Influencing HIT ##

*BETA RELEASE: I should emphasize this project remains in the experimental stage and there are likely many tweaks to come that will improve/change outcomes. I do believe at this stage that the simulation provides good general insights regarding the relationship between various factors and viral spread.*

### Base US Simulation ###

To explore the various concepts of RKnot and viral spread, we'll use a simulation design based on [CDC Best Planning Scenario](https://www.cdc.gov/coronavirus/2019-ncov/hcp/planning-scenarios.html) guidelines for
COVID-19 characteristics including:

* $R_0$ 2.5
* IFR for each of 4 age groups

*Note while sars-cov-2 is obviously a topical globally, the goal of RKnot is to be applicable to any virus (or anything that spreads like a virus). sars-cov-2 is used only for illustration purposes.*

Other assumptions:

* Population of $10,000^1$
* Initial Infected of 2
* Duration of Infectiousness 14 days$^2$
* Duration of Immunity 365 days
* Density of ~1 subject per location (`dlevel='medium'`)

$^1$*proportionately split among the 4 age groups to match US census data.*

$^2$*equal likelihood of transmission on any day (i.e. no viral load curve)*

[US Census Data](https://www.census.gov/prod/cen2010/briefs/c2010br-03.pdf)

### Natural ###

#### 1. Equal ####
The first simulation makes the most homogeneous assumptions. 

+ No group is restricted in terms of movement. 
+ All dots are able to interact with one another. 
+ All dots are susceptible at initiation.
+ All dots are equally likely to move to any dot on the grid (`mover='equal'`)

In [1]:
import matplotlib
matplotlib.use('Qt5Agg')

from matplotlib.animation import FFMpegWriter

In [2]:
group1 = dict(
    name='0-19',
    n=2700,
    n_inf=0,
    ifr=0.00003,
    mover='equal',
)
group2 = dict(
    name='20-49',
    n=4100,
    n_inf=1,
    ifr=0.0002,
    mover='equal',
)
group3 = dict(
    name='50-69',
    n=2300,
    n_inf=1,
    ifr=0.005,
    mover='equal',
)
group4 = dict(
    name='70+',
    n=900,
    n_inf=0,
    ifr=0.054,
    mover='equal',
)
groups = [group1, group2, group3, group4]
params = {'dlevel': 'med', 'Ro':2.5, 'days': 365, 'imndur': 365, 'infdur': 14}

In [3]:
from rknot.sim import Sim

sim = Sim(groups=groups, **params)
sim.run()

2020-10-06 11:14:38,049	INFO resource_spec.py:223 -- Starting Ray with 11.28 GiB memory available for workers and up to 5.66 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-10-06 11:14:38,588	INFO services.py:1191 -- View the Ray dashboard at localhost:8265


---------------------------------------------------------------------------------
|                                  SIM DETAILS                                  |
|-------------------------------------------------------------------------------|
|           boundary|  [  1 100   1 100]|             n_locs|              10000|
|-------------------|-------------------|-------------------|-------------------|
|                  n|              10000|            density|                1.0|
|-------------------|-------------------|-------------------|-------------------|
|                ktr|                1.0|                tmr|               0.18|
|-------------------|-------------------|-------------------|-------------------|


Running the animation will result in a video as per below:

In [4]:
from rknot.animate import SimChart

chart = SimChart(sim)
chart.animate.to_html5_video()

In [5]:
# chart.animate.save('vids/baseus/1.mp4', writer=FFMpegWriter(fps=10))

<video width='705' height='340' 
    src='https://storage.googleapis.com/rknotvids/baseus/1.mp4' controls>alternative text</video>

*Note embedded videos are used for convenience purposes. Given the random processes involved, running the same code will produce slightly different results each time.*

The sim results in a relatively standard spread curve with 

|         |            |
| ------------- |:-------------:|
| Peak      |   46% |
| HIT      | 70%      |
| Fatalities | 0.7%      |

You'll note this is higher than the expected HIT of 60% for $R_0$ 2.5. The differences result given:

1. The simulation is a stochastic process and random variation will result in variation from the mathematical model in smaller populations.
2. This is not an entirely homogeneous population, with different IFRs and different numbers of contacts between subjects. In particular, because dots can occupy the same location at the same time, and [multiple infected dots at the same location increase the transmission rate for any susceptible](intro.ipynb#Likelihood-of-Transmission), this tends to lead to faster spread than $R_0$ would suggest.

#### 2. Local ####
In remaining simulations, we begin to introduce ever increasing homogeneity.

Our first change is to adjust the subjects mover functions to `local`. The `local` mover has a strong bias towards locations only in its immediate vicinity, which is a better approximation of real world processes (though certainly not a perfect one).

In [6]:
group1['mover'] = 'local'
group2['mover'] = 'local'
group3['mover'] = 'local'
group4['mover'] = 'local'
groups = [group1, group2, group3, group4]

In [7]:
sim = Sim(groups=groups, **params)
sim.run()

chart = SimChart(sim)
chart.animate.to_html5_video()

2020-10-06 11:14:42,601	INFO resource_spec.py:223 -- Starting Ray with 11.47 GiB memory available for workers and up to 5.76 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-10-06 11:14:43,110	INFO services.py:1191 -- View the Ray dashboard at localhost:8265


---------------------------------------------------------------------------------
|                                  SIM DETAILS                                  |
|-------------------------------------------------------------------------------|
|           boundary|  [  1 100   1 100]|             n_locs|              10000|
|-------------------|-------------------|-------------------|-------------------|
|                  n|              10000|            density|                1.0|
|-------------------|-------------------|-------------------|-------------------|
|                ktr|                1.0|                tmr|               0.18|
|-------------------|-------------------|-------------------|-------------------|


In [8]:
# chart.animate.save('vids/baseus/2.mp4', writer=FFMpegWriter(fps=10))

<video width='705' height='340' 
    src='https://storage.googleapis.com/rknotvids/baseus/2.mp4' controls>Video Failed To Load</video>

This results in:

|         |            |
| ------------- |:-------------:|
| Peak      |   5% |
| HIT      | 30%      |
| Fatalities | 0.6%      |

We can see that restricting movement has a major impact on limiting spread, however, the virus does remain among the population and result in high penetration and relatively high deaths over time.

*CAUTION: RKnot estimates transmission rate assuming all subjects are* `equal` *movers. Thus, where* `local` *movers are used, the transmission rate may (or may not) be underestimated. This is being investigated.*

#### 3. Social Millenials ####
In this simulation, we set `mover=social` for just the 20-49 age group. This is a rough approximation of that group's real-world propensity to travel more frequently (or go to more events).

In [9]:
group1['mover'] = 'local'
group2['mover'] = 'social'
group3['mover'] = 'local'
group4['mover'] = 'local'
groups = [group1, group2, group3, group4]

In [10]:
sim = Sim(groups=groups, **params)
sim.run()

chart = SimChart(sim)
chart.animate.to_html5_video()

2020-10-06 11:14:46,833	INFO resource_spec.py:223 -- Starting Ray with 11.33 GiB memory available for workers and up to 5.68 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-10-06 11:14:47,362	INFO services.py:1191 -- View the Ray dashboard at localhost:8265


---------------------------------------------------------------------------------
|                                  SIM DETAILS                                  |
|-------------------------------------------------------------------------------|
|           boundary|  [  1 100   1 100]|             n_locs|              10000|
|-------------------|-------------------|-------------------|-------------------|
|                  n|              10000|            density|                1.0|
|-------------------|-------------------|-------------------|-------------------|
|                ktr|                1.0|                tmr|               0.18|
|-------------------|-------------------|-------------------|-------------------|


In [11]:
# chart.animate.save('vids/baseus/3.mp4', writer=FFMpegWriter(fps=10))

<video width='705' height='340' 
    src='https://storage.googleapis.com/rknotvids/baseus/3.mp4' controls>Video Failed To Load</video>

This results in:

|         |            |
| ------------- |:-------------:|
| Peak      |   40% |
| HIT      | 65%      |
| Fatalities | 0.5%      |

We can see how powerful mixing is within a population. Even with the majority of subjects moving only locally, if a large enough population is moving broadly across the space, it will significantly increase the rate of spread.

#### 4. Pre-Existing Immunity ####
In this scenario, we adjust the susceptibility factor for just two groups by relatively small amounts as follows:

+ 20-49: 80%
+ 50-69: 65%

This means, in the inverse, that 10% and 25% of the subjects in the respective groups are already immune to the virus (whether through pre-existing T-cell immunity, anti-bodies, or otherwise).

The older group is assumed to have a lower susceptibility factor as it is more likely that older people will have had more exposure to similar viruses over their lifetime.

T-cell immunity to sars-cov-2 remains a controversial subject, but [many studies](https://www.bmj.com/content/370/bmj.m3563) have found prevalance of T-cells between 20% - 50% in *people unexposed to sars-cov-2*. It is suggested that exposure to "common cold" coronaviruses (or more dangerous ones) may convey this immunity.

In [12]:
group2['susf'] = .8
group3['susf'] = .65
groups = [group1, group2, group3, group4]
sim = Sim(groups=groups, **params)
sim.run()

chart = SimChart(sim)
chart.animate.to_html5_video()

In [13]:
# chart.animate.save('vids/baseus/4.mp4', writer=FFMpegWriter(fps=10))

<video width='705' height='340'
    src='https://storage.googleapis.com/rknotvids/baseus/4.mp4' controls>Video Failed to Load</video>

This results in:

|         |            |
| ------------- |:-------------:|
| Peak      |   22% |
| HIT      | 41%      |
| Fatalities | 0.4%      |

We can see that pre-existing immunity would significantly reduce HIT, but perhaps not impact the fatality rate. This is likely because, in this Sim, the elderly are assumed to *not* have pre-existing immunity.

#### 5. Events ####
Certainly, the vast majority of people in the US do not move in such pre-defined ways as set out by the `mover` function. In reality, people tend to move with a local bias with a small number of interactions, supplemented by larger movements to locations with a large number of interactions in a small amount of time.

In RKnot, we can simulate this with Event objects. And we will incorporate a number of them in this simulation. 

First, we will reset the susceptibility factors of all the groups to 1 and reset all `mover` functions to `local`.

In [14]:
group2['mover'] = 'local'
group2['susf'] = 1
group3['susf'] = 1
groups = [group1, group2, group3, group4]

Then we will instantiate a handful of events recurring periodically over the duration of the sim.

In [15]:
from rknot.events import Event

school1 = Event(name='school1', xy=[25,42], start_tick=2, groups=[0], capacity=10, recurring=2)
school2 = Event(name='school2', xy=[78,82], start_tick=3, groups=[0], capacity=10, recurring=2)
game = Event(name='game', xy=[50,84], start_tick=6, groups=[0,1,2], capacity=100, recurring=14)
concert1 = Event(name='concert1', xy=[20,20], start_tick=7, groups=[1], capacity=50, recurring=14)                 
concert2 = Event(name='concert2', xy=[91,92], start_tick=21, groups=[1], capacity=50, recurring=14)
bar = Event(name='bar', xy=[17,24], start_tick=4, groups=[1], capacity=5, recurring=7)
bar2 = Event(name='bar2', xy=[87,13], start_tick=5, groups=[1], capacity=5, recurring=7)
bar3 = Event(name='bar3', xy=[52,89], start_tick=6, groups=[1,2], capacity=5, recurring=7)
church = Event(name='church', xy=[2,91], start_tick=7, groups=[2,3], capacity=20, recurring=7)

events = [school1, school2, game, concert1, concert2, bar, bar2, bar3, church]

In [16]:
sim = Sim(groups=groups, events=events, **params)
sim.run()
chart = SimChart(sim)
chart.animate.to_html5_video()

2020-10-06 11:14:51,065	INFO resource_spec.py:223 -- Starting Ray with 11.23 GiB memory available for workers and up to 5.64 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-10-06 11:14:51,559	INFO services.py:1191 -- View the Ray dashboard at localhost:8265


---------------------------------------------------------------------------------
|                                  SIM DETAILS                                  |
|-------------------------------------------------------------------------------|
|           boundary|  [  1 100   1 100]|             n_locs|              10000|
|-------------------|-------------------|-------------------|-------------------|
|                  n|              10000|            density|                1.0|
|-------------------|-------------------|-------------------|-------------------|
|                ktr|                1.0|                tmr|               0.18|
|-------------------|-------------------|-------------------|-------------------|


In [17]:
# chart.animate.save('vids/baseus/5.mp4', writer=FFMpegWriter(fps=10))

<video width='705' height='340'
    src='https://storage.googleapis.com/rknotvids/baseus/5.mp4' controls>Video Failed to Load</video>

This results in:

|         |            |
| ------------- |:-------------:|
| Peak      |   18% |
| HIT      | 49%      |
| Fatalities | 0.5%      |

We can see that this sort of event-based structure resulted in lower peak infections and HIT relative to Example 3. The curve is flatter and longer. 

The selection of events here is relatively arbitrary and perhaps not perfectly illustrative of the nature of movement but it does highlight how more heterogeneous movement might lead to lower HIT.

#### 6. Gates ####
We can further improve the real world interactions by introducing gates. Subjects are not always freely able to interact with all other people in a population. Often their movement is restricted to within certain areas. Furthermore, other people's access into those areas is restricted.

Elderly people living in retirement homes or assisted living centers is an example. To simulate this, we will split `group4` into two separate groups. 

* `group4a`
    * population of 600 (2/3s of `group4`) 
    * IFR of 4.2%
    * move freely throughout the entire grid as previously
* `group4b` 
    * population of 300 (1/3rd of `group4`)
    * IFR of 7.8%
    * movement restricted to 6x6 box

We have also adjusted the IFR on the basis that `group4b` is likely older and also probably more frail than `group4a`. IFRs approximate those found [here](https://www.publichealthontario.ca/-/media/documents/ncov/epi/2020/06/covid19-epi-case-identification-age-only-template.pdf?la=en).

In addition, we will update certain events to account for the additional group including adding an event specifically for the new group.

In [18]:
group4a = dict(
    name='70+',
    n=600,
    n_inf=0,
    ifr=0.042,
    mover='local',
)
group4b = dict(
    name='70+G',
    n=300,
    n_inf=0,
    ifr=0.0683,
    mover='local',
    box=[1,6,1,6],
    box_is_gated=True,
)
groups = [group1, group2, group3, group4a, group4b]
game = Event(name='game', xy=[50,84], start_tick=6, groups=[0,1,2,3], capacity=100, recurring=14)
church = Event(name='church', xy=[2,91], start_tick=7, groups=[2,3], capacity=20, recurring=7)
church2 = Event(name='church', xy=[2,3], start_tick=7, groups=[4], capacity=5, recurring=7)
events = [game, concert1, concert2, bar, bar2, bar3, church, church2]

Now, such elderly populations are not totally sealed of from the rest of the world. In fact, they are often visited by family members or friends. We can mimick this with the use of a Travel object.

In this sim, at least one person will enter into the `group4b` gate for a day only. And this will repeat every day of the sim.

In [19]:
from rknot.events import Travel
visit = Travel(name='visit', xy=[1,1], start_tick=3, groups=[1,2], capacity=1, duration=1, recurring=1)
events.append(visit)

In [20]:
sim = Sim(groups=groups, events=events, **params)
sim.run()
chart = SimChart(sim)
chart.animate.to_html5_video()

2020-10-06 11:14:55,389	INFO resource_spec.py:223 -- Starting Ray with 11.23 GiB memory available for workers and up to 5.62 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-10-06 11:14:55,911	INFO services.py:1191 -- View the Ray dashboard at localhost:8265


---------------------------------------------------------------------------------
|                                  SIM DETAILS                                  |
|-------------------------------------------------------------------------------|
|           boundary|  [  1 100   1 100]|             n_locs|              10000|
|-------------------|-------------------|-------------------|-------------------|
|                  n|              10000|            density|                1.0|
|-------------------|-------------------|-------------------|-------------------|
|                ktr|                1.0|                tmr|               0.18|
|-------------------|-------------------|-------------------|-------------------|


In [21]:
# chart.animate.save('vids/baseus/6.mp4', writer=FFMpegWriter(fps=10))

<video width='705' height='340'
    src='https://storage.googleapis.com/rknotvids/baseus/6.mp4' controls>Video Failed to Load</video>

This results in:

|         |            |
| ------------- |:-------------:|
| Peak      |   26% |
| HIT      | 60%      |
| Fatalities | 0.5%      |

This gated structure results in essentially no impact. The use for this structure will become more evident in the disucssion on [Policy Decisions.](#Policy)

#### 7. Events, Gates, and Pre-Immunity ####
Now we can see how pre-immunity might impact viral spread in a population with more homogeneous interactions.

We will also split `group2` into two separate groups with different movers (`group2a` and `group2b`. The 20-49 age group is a large cohort and not all members will move in the same way. Some will move socially, other more locally.

Then, we update the group list and update our events for the new group structure.

In [22]:
group2['susf'] = .8
group3['susf'] = .65
groups = [group1, group2, group3, group4a, group4b]

In [23]:
game = Event(name='game', xy=[50,84], start_tick=6, groups=[0,1,2], capacity=100, recurring=14)
concert1 = Event(name='concert1', xy=[20,20], start_tick=7, groups=[1], capacity=50, recurring=14)                 
concert2 = Event(name='concert2', xy=[91,92], start_tick=21, groups=[1], capacity=50, recurring=14)
bar = Event(name='bar', xy=[17,24], start_tick=4, groups=[1], capacity=5, recurring=7)
bar2 = Event(name='bar2', xy=[87,13], start_tick=5, groups=[1], capacity=5, recurring=7)
bar3 = Event(name='bar3', xy=[52,89], start_tick=6, groups=[1,2], capacity=5, recurring=7)
church = Event(name='church', xy=[2,91], start_tick=7, groups=[2,3], capacity=20, recurring=7)
church2 = Event(name='church', xy=[2,3], start_tick=7, groups=[4], capacity=5, recurring=7)

visit = Travel(name='visit', xy=[1,1], start_tick=3, groups=[1,2], capacity=1, duration=1, recurring=1)

events = [
    school1, school2,
    game, concert1, concert2, bar, bar2, bar3, 
    church, church2, visit,
]

In [24]:
sim = Sim(groups=groups, events=events, **params)
sim.run()
chart = SimChart(sim)
chart.animate.to_html5_video()

2020-10-06 11:14:59,751	INFO resource_spec.py:223 -- Starting Ray with 11.18 GiB memory available for workers and up to 5.59 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-10-06 11:15:00,249	INFO services.py:1191 -- View the Ray dashboard at localhost:8265


---------------------------------------------------------------------------------
|                                  SIM DETAILS                                  |
|-------------------------------------------------------------------------------|
|           boundary|  [  1 100   1 100]|             n_locs|              10000|
|-------------------|-------------------|-------------------|-------------------|
|                  n|              10000|            density|                1.0|
|-------------------|-------------------|-------------------|-------------------|
|                ktr|                1.0|                tmr|               0.18|
|-------------------|-------------------|-------------------|-------------------|


In [25]:
# chart.animate.save('vids/baseus/7.mp4', writer=FFMpegWriter(fps=10))

<video width='705' height='340'
    src='https://storage.googleapis.com/rknotvids/baseus/7.mp4' controls>Video Failed to Load</video>

This results in:

|         |            |
| ------------- |:-------------:|
| Peak      |   16% |
| HIT      | 33%      |
| Fatalities | 0.4%      |

So once again we see that pre-existing immunity would have the effect of significanlty reducing peak infections, HIT, and fatalities. Also note the longer tail that results compared to Example 4 (the flatter the curve, the longer the tail).

#### 8. Self Aware Social Distancing ####
In a self-aware population, we can also incorporate an assumption that certain members of the population will implement social distancing practices (even in the absence of prescribed government policy). For example, individuals might wear masks or face shields while in public. 

This is implemented via a SocialDistancing object, which reduces the transmission factor of the subjects in the applicable group.

In [26]:
from rknot.events import SocialDistancing as SD

sd = SD(name='6-feet', tmfs=[.975,.95,.75,.5], groups=[1,2,3,4], start_tick=5, duration=90)

events.append(sd)

In [27]:
sim = Sim(groups=groups, events=events, **params)
sim.run()
chart = SimChart(sim)
chart.animate.to_html5_video()

2020-10-06 11:15:03,827	INFO resource_spec.py:223 -- Starting Ray with 11.13 GiB memory available for workers and up to 5.58 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-10-06 11:15:04,318	INFO services.py:1191 -- View the Ray dashboard at localhost:8265


---------------------------------------------------------------------------------
|                                  SIM DETAILS                                  |
|-------------------------------------------------------------------------------|
|           boundary|  [  1 100   1 100]|             n_locs|              10000|
|-------------------|-------------------|-------------------|-------------------|
|                  n|              10000|            density|                1.0|
|-------------------|-------------------|-------------------|-------------------|
|                ktr|                1.0|                tmr|               0.18|
|-------------------|-------------------|-------------------|-------------------|


In [28]:
# chart.animate.save('vids/baseus/8.mp4', writer=FFMpegWriter(fps=10))

<video width='705' height='340'
    src='https://storage.googleapis.com/rknotvids/baseus/8.mp4' controls>Video Failed to Load</video>

This results in:

|         |            |
| ------------- |:-------------:|
| Peak      |   12% |
| HIT      | 34%      |
| Fatalities | 0.2%      |

Here we can that social distancing in a population does have a modest impact on peak infections. We also can see that social distancing among the most susceptible (i.e. `group4b`) was a key factor in a dramatically reduced fatality rate (50-75% reduction in fatalities).

Essentially, in this simulation, social distancing prevented an outbreak in the `group4b` community (whereas outbreaks had previously occured there in prior sims).

*Note that randomness in an important factor in viral transmission, so the results of the sim above will not be repeatable every time, though on balance we would expect this outcome to occur more often than in say, Example 7.*

#### 9. Isolation ####
TBD

#### 10. Infectiousness Curve ####
TBD

### Policy ###

With a more realistic model of subject interaction, we can begin to experiment with the impact of different policy measures.

RKnot can simulate policy measures via `Restriction`, `SocialDistancing`, and `Quarantine` objects.

First we will reset our groups. We will maintain the 5 group structure with 1/3rd of the 70+ population gated. No groups will have pre-existing immunity. Remaining characteristics are as [per here](#).

Events are as [per here](#).

In [29]:
group1 = dict(
    name='0-19',
    n=2700,
    n_inf=0,
    ifr=0.00003,
    mover='local',
)
group2 = dict(
    name='20-49',
    n=4100,
    n_inf=1,
    ifr=0.0002,
    mover='local',
)
group3 = dict(
    name='50-69',
    n=2300,
    n_inf=1,
    ifr=0.005,
    mover='local',
)
group4a = dict(
    name='70+',
    n=600,
    n_inf=0,
    ifr=0.042,
    mover='local',
)
group4b = dict(
    name='70+G',
    n=300,
    n_inf=0,
    ifr=0.0683,
    mover='local',
    box=[1,6,1,6],
    box_is_gated=True,
)
groups = [group1, group2, group3, group4a, group4b]
params = {'dlevel': 'med', 'Ro':2.5, 'days': 365, 'imndur': 365, 'infdur': 14}

In [30]:
from rknot.events import Event, Travel

school1 = Event(name='school1', xy=[25,42], start_tick=2, groups=[0], capacity=10, recurring=2)
school2 = Event(name='school2', xy=[78,82], start_tick=3, groups=[0], capacity=10, recurring=2)
game = Event(name='game', xy=[50,84], start_tick=6, groups=[0,1,2], capacity=100, recurring=14)
concert1 = Event(name='concert1', xy=[20,20], start_tick=7, groups=[1], capacity=50, recurring=14)                 
concert2 = Event(name='concert2', xy=[91,92], start_tick=21, groups=[1], capacity=50, recurring=14)
bar = Event(name='bar', xy=[17,24], start_tick=4, groups=[1], capacity=5, recurring=7)
bar2 = Event(name='bar2', xy=[87,13], start_tick=5, groups=[1], capacity=5, recurring=7)
bar3 = Event(name='bar3', xy=[52,89], start_tick=6, groups=[1,2], capacity=5, recurring=7)
church = Event(name='church', xy=[2,91], start_tick=7, groups=[2,3], capacity=20, recurring=7)
church2 = Event(name='church', xy=[2,3], start_tick=7, groups=[4], capacity=5, recurring=7)

visit = Travel(name='visit', xy=[1,1], start_tick=3, groups=[1,2], capacity=1, duration=1, recurring=1)

events = [
    school1, school2,
    game, concert1, concert2, bar, bar2, bar3, 
    church, church2, visit,
]

#### 1. Restrict Large Gatherings ####
We'll start by simply restricting large gathers, which for this sim is any event with 10+ capacity (0.1% of the entire population). The restrictions will last for 60 days.

When considering capacity, remember that a 100,000-seat stadium in a 10 million person catchment represents 1% of the population.

We assume this policy are implemented on day 30, when the population finally realizes there is a pandemic and government has had time to implement prevention measures.

The restriction will last for 60 days.

In [31]:
from rknot.events import Restriction

large_gatherings = Restriction(
    name='large', start_tick=30, duration=60, criteria={'capacity': 10}
)
events_w_res = events + [large_gatherings]

In [32]:
sim = Sim(groups=groups, events=events_w_res, **params)
sim.run()
chart = SimChart(sim)
chart.animate.to_html5_video()

2020-10-06 11:15:07,855	INFO resource_spec.py:223 -- Starting Ray with 11.13 GiB memory available for workers and up to 5.59 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-10-06 11:15:08,351	INFO services.py:1191 -- View the Ray dashboard at localhost:8265


---------------------------------------------------------------------------------
|                                  SIM DETAILS                                  |
|-------------------------------------------------------------------------------|
|           boundary|  [  1 100   1 100]|             n_locs|              10000|
|-------------------|-------------------|-------------------|-------------------|
|                  n|              10000|            density|                1.0|
|-------------------|-------------------|-------------------|-------------------|
|                ktr|                1.0|                tmr|               0.18|
|-------------------|-------------------|-------------------|-------------------|


In [33]:
# chart.animate.save('vids/baseus/large.mp4', writer=FFMpegWriter(fps=10))

<video width='705' height='340'
    src='https://storage.googleapis.com/rknotvids/baseus/large.mp4' controls>Video Failed to Load</video>


This results in:

|         |            |
| ------------- |:-------------:|
| Peak      |   25% |
| HIT      | 56%      |
| Fatalities | 0.6%      |

We see that the curve is flattened significanlty during the restriction period and the peak of infections has been pushed way out to ~125+ days, but, as restrictions are lifted, the outbreak ensues and fatalities are actually higher than in many other sims.

#### 2. Restrict Elderly Visits ####
We can restrict events by name by passing `name` key to `criteria`. We can do this to restrict the travel event to the 70+ gated area. 

The policy measure is implemented on day 30 and maintained for another 120 days. There will be no other restrictions.

In [3]:
no_visits = Restriction(
    name='no_visits', start_tick=30, duration=120, criteria={'name': 'visit'}
)
events_w_res = events + [no_visits]

In [35]:
sim = Sim(groups=groups, events=events_w_res, **params)
sim.run()
chart = SimChart(sim)
chart.animate.to_html5_video()

2020-10-06 11:15:12,250	INFO resource_spec.py:223 -- Starting Ray with 11.08 GiB memory available for workers and up to 5.56 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-10-06 11:15:12,741	INFO services.py:1191 -- View the Ray dashboard at localhost:8265


---------------------------------------------------------------------------------
|                                  SIM DETAILS                                  |
|-------------------------------------------------------------------------------|
|           boundary|  [  1 100   1 100]|             n_locs|              10000|
|-------------------|-------------------|-------------------|-------------------|
|                  n|              10000|            density|                1.0|
|-------------------|-------------------|-------------------|-------------------|
|                ktr|                1.0|                tmr|               0.18|
|-------------------|-------------------|-------------------|-------------------|


In [36]:
# chart.animate.save('vids/baseus/no_visits.mp4', writer=FFMpegWriter(fps=10))

<video width='705' height='340' 
    src='https://storage.googleapis.com/rknotvids/baseus/no_visits.mp4' controls>Video Failed to Load</video>

This results in:

|         |            |
| ------------- |:-------------:|
| Peak      |   25% |
| HIT      | 54%      |
| Fatalities | 0.3%      |

While this approach does not have a big impact on viral spread, it is the *only policy measure that reduces fatalities in the long run*. The most susceptible are protected during the peak and the virus has "burned out" fast enough that the elderly are not infected when visits resume.

#### 3. Social Distancing ####
We can mimick the implementation of Social Distancing policies in certain settings via the `SocialDistancing` object. Mask wearing, hand sanitizing, and 6-foot perimeters all provide varying levels of protection.

It is hard to estimate the degree of protection from each, and even harder in combination. For instance, [this study](https://pubmed.ncbi.nlm.nih.gov/23498357/) found anywhere between a 1.1- and 55-fold reduction in exposure to influenza with varying mask designs.

So we provide `tmfs` here for illustration purposes only, attempting to catch all social distancing practices. We also provide different `tmfs` for the different age groups, meant to simulate adherence to policy.

The policy measure is implemented on day 30 and maintained for 120 days.

In [37]:
from rknot.events import SocialDistancing as SD

sd = SD(name='all', tmfs=[.8,.8,.7,.65,.5], groups=[0,1,2,3,4], start_tick=30, duration=120)
events_w_res = events + [sd]

In [38]:
sim = Sim(groups=groups, events=events_w_res, **params)
sim.run()
chart = SimChart(sim)
chart.animate.to_html5_video()

2020-10-06 11:15:16,375	INFO resource_spec.py:223 -- Starting Ray with 11.13 GiB memory available for workers and up to 5.57 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-10-06 11:15:16,872	INFO services.py:1191 -- View the Ray dashboard at localhost:8265


---------------------------------------------------------------------------------
|                                  SIM DETAILS                                  |
|-------------------------------------------------------------------------------|
|           boundary|  [  1 100   1 100]|             n_locs|              10000|
|-------------------|-------------------|-------------------|-------------------|
|                  n|              10000|            density|                1.0|
|-------------------|-------------------|-------------------|-------------------|
|                ktr|                1.0|                tmr|               0.18|
|-------------------|-------------------|-------------------|-------------------|


In [39]:
# chart.animate.save('vids/baseus/policy_sd.mp4', writer=FFMpegWriter(fps=10))

<video width='705' height='340'
       src='https://storage.googleapis.com/rknotvids/baseus/policy_sd.mp4' controls>Video Failed to Load</video>

This results in:

|         |            |
| ------------- |:-------------:|
| Peak      |   11% |
| HIT      | 59%      |
| Fatalities | 0.4%      |

We can see that social distancing is certainly the most effective approach (of the last 3) to "flattening the curve" with spread remarkably limited during the policy period, reaching a peak only after ~175+ days. 

Still, once restrictions are lifted the virus spreads unabated and fatalties reach relatively high levels. We can also see the "second wave" effect where infections appear to peak around 100 days, only to rebound and reach a higher peak two months later.

#### 4. Social Distancing and Restrict Elderly Visits and Large Gatherings ####
What happens if we combine the three approaches above?

In [40]:
large_gatherings = Restriction(
    name='large', start_tick=30, duration=60, criteria={'capacity': 10}
)
no_visits = Restriction(
    name='no_visits', start_tick=30, duration=120, criteria={'name': 'visit'}
)
sd = SD(name='all', tmfs=[.8,.8,.7,.65,.5], groups=[0,1,2,3,4], start_tick=30, duration=120)
events_w_res = events + [large_gatherings, no_visits, sd]

In [41]:
sim = Sim(groups=groups, events=events_w_res, **params)
sim.run()
chart = SimChart(sim)
chart.animate.to_html5_video()

2020-10-06 11:15:20,474	INFO resource_spec.py:223 -- Starting Ray with 11.13 GiB memory available for workers and up to 5.58 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-10-06 11:15:20,970	INFO services.py:1191 -- View the Ray dashboard at localhost:8265


---------------------------------------------------------------------------------
|                                  SIM DETAILS                                  |
|-------------------------------------------------------------------------------|
|           boundary|  [  1 100   1 100]|             n_locs|              10000|
|-------------------|-------------------|-------------------|-------------------|
|                  n|              10000|            density|                1.0|
|-------------------|-------------------|-------------------|-------------------|
|                ktr|                1.0|                tmr|               0.18|
|-------------------|-------------------|-------------------|-------------------|


In [42]:
# chart.animate.save('vids/baseus/sd_&_no_large_&_no_visits.mp4', writer=FFMpegWriter(fps=10))

<video width='705' height='340'
    src='https://storage.googleapis.com/rknotvids/baseus/sd_%26_no_large_%26_no_visits.mp4' controls>Video Failed to Load</video>

This results in:

|         |            |
| ------------- |:-------------:|
| Peak      |   9% |
| HIT      | 14%      |
| Fatalities | 0.4%      |

We can see that this combined approach results in one of the flattest curves yet with the virus slowly declining in a very long tail.

Importantly, however, this does not result in signficantly lower fatalities as an outbreak occurs in the `group4b` gate just before the visit restriction kicks on day 30. 

Now is an important time to reiterate that there can be significant variance between simulations run with the same structure. Here is the same simulation, run a second time:



In [43]:
# chart.animate.save('vids/baseus/sd_&_no_large_&_no_visits_2.mp4', writer=FFMpegWriter(fps=10))

<video width='705' height='340'
   src='https://storage.googleapis.com/rknotvids/baseus/sd_%26_no_large_%26_no_visits_2.mp4' controls>Video Failed to Load</video>

This results in:

|         |            |
| ------------- |:-------------:|
| Peak      |   22% |
| HIT      | 54%      |
| Fatalities | 0.5%      |

In this sim, the measures are almost successful in eradicating the virus, as the infection level cruises at near zero for 160+ days, then the restrictions are lifted in their entirety and virus spreads throughtout the population.

`group4b` manages to avoid an outbreak in the early days, but, once travel restrictions are lifted, an otubreak occurs almost immediately, leading to a high level of fatalities.

#### 5. Quarantine ####
We can even mimick the impact of quarantines via the Quarantine object.

Here we show the impact of a 30-day quarantine for all groups in the Sim.

In [44]:
from rknot.events import Quarantine

quarantine = Quarantine(name='all', start_tick=30, groups=[0,1,2,3,4], duration=30)
events_w_res = events + [quarantine]

In [45]:
sim = Sim(groups=groups, events=events_w_res, **params)
sim.run()
chart = SimChart(sim)
chart.animate.to_html5_video()

2020-10-06 11:15:24,744	INFO resource_spec.py:223 -- Starting Ray with 11.08 GiB memory available for workers and up to 5.56 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-10-06 11:15:25,232	INFO services.py:1191 -- View the Ray dashboard at localhost:8265


---------------------------------------------------------------------------------
|                                  SIM DETAILS                                  |
|-------------------------------------------------------------------------------|
|           boundary|  [  1 100   1 100]|             n_locs|              10000|
|-------------------|-------------------|-------------------|-------------------|
|                  n|              10000|            density|                1.0|
|-------------------|-------------------|-------------------|-------------------|
|                ktr|                1.0|                tmr|               0.18|
|-------------------|-------------------|-------------------|-------------------|


In [46]:
# chart.animate.save('vids/baseus/quarantine.mp4', writer=FFMpegWriter(fps=10))

<video width='705' height='340'
       src='https://storage.googleapis.com/rknotvids/baseus/quarantine.mp4' controls>Video Failed to Load</video>

This results in:

|         |            |
| ------------- |:-------------:|
| Peak      |   24% |
| HIT      | 61%      |
| Fatalities | 0.4%      |

The quarantine measures are effective in suppressing spread during the quarantine, but viral spread continues once the measures are lifted.

#### 6. Quarantine Then Restrictions ####
We can combine a quarantine with following on policy restrictions to mimick the decisions made by many governments worldwide.

To do this, we will push the start_tick of the restrictions out until the end of the quarantine (quarantines and restrictions cannot overlap).

In [47]:
large_gatherings = Restriction(
    name='large', start_tick=61, duration=60, criteria={'capacity': 10}
)
no_visits = Restriction(
    name='no_visits', start_tick=61, duration=120, criteria={'name': 'visit'}
)
sd = SD(name='all', tmfs=[.8,.8,.7,.65,.5], groups=[0,1,2,3,4], start_tick=61, duration=120)
quarantine = Quarantine(name='all', start_tick=30, groups=[0,1,2,3,4], duration=30)
events_w_res = events + [large_gatherings, no_visits, sd, quarantine]

In [48]:
sim = Sim(groups=groups, events=events_w_res, **params)
sim.run()
chart = SimChart(sim)
chart.animate.to_html5_video()

2020-10-06 11:15:28,961	INFO resource_spec.py:223 -- Starting Ray with 11.08 GiB memory available for workers and up to 5.56 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-10-06 11:15:29,455	INFO services.py:1191 -- View the Ray dashboard at localhost:8265


---------------------------------------------------------------------------------
|                                  SIM DETAILS                                  |
|-------------------------------------------------------------------------------|
|           boundary|  [  1 100   1 100]|             n_locs|              10000|
|-------------------|-------------------|-------------------|-------------------|
|                  n|              10000|            density|                1.0|
|-------------------|-------------------|-------------------|-------------------|
|                ktr|                1.0|                tmr|               0.18|
|-------------------|-------------------|-------------------|-------------------|


In [49]:
# chart.animate.save('vids/baseus/quarantine_then_restrict.mp4', writer=FFMpegWriter(fps=10))

<video width='705' height='340'
       src='https://storage.googleapis.com/rknotvids/baseus/quarantine_then_restrict.mp4' controls>Video Failed to Load</video>

This results in:

|         |            |
| ------------- |:-------------:|
| Peak      |   1% |
| HIT      | 1%      |
| Fatalities | 0%      |

The pandemic never materializes in this simulation. The combination of policy measures eradicates the virus swiftly. 

This is a very interesting result. This obvioulsy did not occur in many places that took this approach. There are several possible reasons worth considering:

* Random Variation: if you run this simulation multiple times, you will note sims where spread does occur (though fewer than other sims).
* Scale: a larger simulation would increase the possibility that a very small number of dots can continue to pass around the virus while it is largerly muted in the broader population
* Adherence: it is possible that real-world adherence to the implemented policies was much lower than the modeling suggests

### A Fulsome Scenario ###

We'll now create a more fulsome scenario encompassing many of the factors discussed.

Adjustments to prior scenarios include:

* add additional group with `mover='social'` in the 20-49 cohort. this cohort also flaunts social distancing guidelines as so its tmf in the SD object is `1.25`
* add susceptibility factors for most groups
* more fine-tuned capacity restrictions that gradually allow larger events over time
* restrict visits to the elderly for a full year

In [50]:
### GROUPS ###
group1 = dict(
    name='0-19',
    n=2700,
    n_inf=0,
    ifr=0.00003,
    mover='local',
)
group2a = dict(
    name='20-49L',
    n=3900,
    n_inf=1,
    ifr=0.0002,
    mover='local',
    susf=0.85,    
)
group2b = dict(
    name='20-49S',
    n=200,
    n_inf=1,
    ifr=0.0002,
    mover='social',
    susf=0.85,    
)
group3 = dict(
    name='50-69',
    n=2300,
    n_inf=1,
    ifr=0.005,
    susf=.7,
    mover='local',
)
group4a = dict(
    name='70+',
    n=600,
    n_inf=0,
    ifr=0.042,
    mover='local',
    susf=0.9,    
)
group4b = dict(
    name='70+G',
    n=300,
    n_inf=0,
    ifr=0.0683,
    mover='local',
    box=[1,6,1,6],
    box_is_gated=True,
)
groups = [group1, group2a, group2b, group3, group4a, group4b]
params = {'dlevel': 'med', 'Ro':2.5, 'days': 365, 'imndur': 365, 'infdur': 14}

### EVENTS ###
from rknot.events import Event, Travel

school1 = Event(name='school1', xy=[25,42], start_tick=2, groups=[0], capacity=10, recurring=2)
school2 = Event(name='school2', xy=[78,82], start_tick=3, groups=[0], capacity=10, recurring=2)
game = Event(name='game', xy=[50,84], start_tick=6, groups=[0,1,2,3], capacity=100, recurring=14)
concert1 = Event(name='concert1', xy=[20,20], start_tick=7, groups=[1,2], capacity=50, recurring=14)                 
concert2 = Event(name='concert2', xy=[91,92], start_tick=21, groups=[1,2], capacity=50, recurring=14)
bar = Event(name='bar', xy=[17,24], start_tick=4, groups=[1,2], capacity=5, recurring=7)
bar2 = Event(name='bar2', xy=[87,13], start_tick=5, groups=[1,2], capacity=5, recurring=7)
bar3 = Event(name='bar3', xy=[52,89], start_tick=6, groups=[1,2,3], capacity=5, recurring=7)
church = Event(name='church', xy=[2,91], start_tick=7, groups=[3,4], capacity=20, recurring=7)
church2 = Event(name='church', xy=[2,3], start_tick=7, groups=[5], capacity=5, recurring=7)

visit = Travel(name='visit', xy=[1,1], start_tick=3, groups=[1,2,3], capacity=1, duration=1, recurring=1)

events = [
    school1, school2,
    game, concert1, concert2, bar, bar2, bar3, 
    church, church2, visit,
]

### RESTRICTIONS ###
small_gatherings = Restriction(
    name='small', start_tick=30, duration=30, criteria={'capacity': 5}
)
med_gatherings = Restriction(
    name='med', start_tick=60, duration=30, criteria={'capacity': 10}
)
large_gatherings = Restriction(
    name='large', start_tick=90, duration=30, criteria={'capacity': 50}
)
no_visits = Restriction(
    name='no_visits', start_tick=30, duration=365, criteria={'name': 'visit'}
)
sd = SD(name='all', tmfs=[.8, .8, 1.25, .7,.65,.5], groups=[0,1,2,3,4,5], start_tick=30, duration=120)
events_w_res = events + [small_gatherings, med_gatherings, large_gatherings, no_visits, sd]

In [51]:
sim = Sim(groups=groups, events=events_w_res, **params)
sim.run()
chart = SimChart(sim)
chart.animate.to_html5_video()

2020-10-06 11:15:33,293	INFO resource_spec.py:223 -- Starting Ray with 11.08 GiB memory available for workers and up to 5.56 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-10-06 11:15:33,782	INFO services.py:1191 -- View the Ray dashboard at localhost:8265


---------------------------------------------------------------------------------
|                                  SIM DETAILS                                  |
|-------------------------------------------------------------------------------|
|           boundary|  [  1 100   1 100]|             n_locs|              10000|
|-------------------|-------------------|-------------------|-------------------|
|                  n|              10000|            density|                1.0|
|-------------------|-------------------|-------------------|-------------------|
|                ktr|                1.0|                tmr|               0.18|
|-------------------|-------------------|-------------------|-------------------|


In [52]:
# chart.animate.save('vids/baseus/fulsome.mp4', writer=FFMpegWriter(fps=10))

<video width='705' height='340'
       src='https://storage.googleapis.com/rknotvids/baseus/fulsome.mp4' controls>Video Failed to Load</video>

This results in:

|         |            |
| ------------- |:-------------:|
| Peak      |   15% |
| HIT      | 34%      |
| Fatalities | 0.2%      |

This has the lowest fatality rate of the group and is achieved simply by restricting access to gated region 